# Neural Network Classifier

In this notebook I explore building a neural netwrok classifier from lagged returns data. As with the SVM, this model is fundamentally flawed due to the efficient markets hypothesis, but again this provides good practice for building and backtesting models.

I use PyTorch and Lightning here. This is obviously overkill for such a simple NN classifier however, it is once again a good learning experience.

In [11]:
import pandas as pd
import lightning as L
import torch
from torch import nn

In [10]:
class Classifier(nn.Module):
    def __init__(self, input_size, output_size, hidden_l1, hidden_l2):
        super().__init__()
    
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_l1),
            nn.ReLU(),
            nn.Linear(hidden_l1, hidden_l2),
            nn.ReLU(),
            nn.Linear(hidden_l2, output_size),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
class LitClassifier(L.LightningModule):
    def __init__(self, Classifier, learning_rate):
        super().__init__()
        self.Classifier = Classifier
        self.learning_rate= learning_rate
        self.BCE = torch.nn.BCEWithLogitsLoss()

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        # x = x.view(x.size(0), -1)
        x = self.Classifier(x)
        loss = self.BCE(x, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
class MNISTDataModule(L.LightningDataModule):
    def __init__(self, data_dir: str = "path/to/dir", batch_size: int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage: str):
        self.mnist_test = MNIST(self.data_dir, train=False)
        self.mnist_predict = MNIST(self.data_dir, train=False)
        mnist_full = MNIST(self.data_dir, train=True)
        self.mnist_train, self.mnist_val = random_split(
            mnist_full, [55000, 5000], generator=torch.Generator().manual_seed(42)
        )

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.mnist_predict, batch_size=self.batch_size)

    def teardown(self, stage: str):
        # Used to clean-up when the run is finished
        ...

In [ ]:
# model
model= LitClassifier(Classifier(5,1,10,10))

# train model
trainer = L.Trainer()
trainer.fit(model=model, train_dataloaders=train_loader)